<div dir="rtl" align="justify">
  <h2>مقدمه‌ای بر هَشینگ (Hashing)</h2>
  <p><b>هشینگ</b> یک تکنیک اساسی در علوم کامپیوتر برای ذخیره و بازیابی سریع اطلاعات است. ایده اصلی این است که یک «جدول هش» (Hash Table) داشته باشیم که در واقع یک آرایه است. ما یک «تابع هش» (Hash Function) داریم که یک کلید (Key) منحصربه‌فرد (مانند یک شناسه) را می‌گیرد و آن را به یک ایندکس (Index) در آرایه تبدیل می‌کند.</p>
  <p>در این مثال، ما یک سناریوی مشخص را بررسی می‌کنیم:</p>
  <ul>
    <li><b>جدول هش:</b> آرایه‌ای با اندازه 101 (ایندکس‌ها از 0 تا 100) است.</li>
    <li><b>کلیدها (Keys):</b> مقادیر عددی هستند (مانند 145، 17، 30، ...).</li>
    <li><b>تابع هش (Hash Function):</b> <code>key % 101</code> (باقیمانده تقسیم کلید بر 101).</li>
  </ul>
  <p>برای مثال، کلید 145 از طریق تابع هش به ایندکس <code>44 = 145 % 101</code> نگاشت می‌شود. این به ما اجازه می‌دهد تا رکورد R1 (مربوط به کلید 145) را مستقیماً در خانه 44 جدول ذخیره کنیم یا پیدا کنیم. این کار در حالت ایده‌آل، جستجو، درج و حذف را در زمان ثابت <i>O(1)</i> ممکن می‌سازد.</p>
</div>

<div dir="rtl" align="justify">
  <h2>مشکل برخورد (Collision) و روش حل: زنجیره‌سازی جداگانه (Separate Chaining)</h2>
  <p>مشکل زمانی رخ می‌دهد که دو یا چند کلید مختلف، به یک ایندکس یکسان هش شوند. به این حالت <b>«برخورد» (Collision)</b> می‌گویند.</p>
  <p>در مثال ما:</p>
  <ul>
    <li><code>h(145) = 145 % 101 = 44</code></li>
    <li><code>h(44)  =  44 % 101 = 44</code></li>
  </ul>
  <p>هر دو کلید 145 (مربوط به R1) و 44 (مربوط به R5) می‌خواهند در ایندکس 44 جدول قرار بگیرند. به همین ترتیب:</p>
  <ul>
    <li><code>h(17)  =  17 % 101 = 17</code></li>
    <li><code>h(118) = 118 % 101 = 17</code></li>
  </ul>
  <p>کلیدهای 17 (R2) و 118 (R6) نیز در ایندکس 17 برخورد دارند.</p>
  <p><b>راه حل: زنجیره‌سازی جداگانه (Separate Chaining)</b>. در این روش، هر خانه از جدول هش (آرایه اصلی) به جای ذخیره خودِ داده، یک <b>اشاره‌گر (Pointer)</b> به ابتدای یک <b>لیست پیوندی (Linked List)</b> را نگه می‌دارد. وقتی برخوردی رخ می‌دهد، رکورد جدید به انتهای آن لیست پیوندی اضافه می‌شود.</p>
  <p>در این پیاده‌سازی:</p>
  <ul>
    <li>ایندکس 17 به لیستی اشاره می‌کند که شامل <code>R2</code> و سپس <code>R6</code> است. (<code>R2 -> R6 -> X</code>)</li>
    <li>ایندکس 44 به لیستی اشاره می‌کند که شامل <code>R1</code> و سپس <code>R5</code> است. (<code>R1 -> R5 -> X</code>)</li>
  </ul>
</div>

<div dir="rtl" align="justify">
  <h2>پیاده‌سازی در پایتون: بخش اول - کلاس گره (Node)</h2>
  <p>ابتدا، ما به کلاسی برای نمایش هر گره (Node) در لیست پیوندی نیاز داریم. هر گره باید خودِ <b>کلید</b> (key)، <b>داده</b> (data) مرتبط با آن (مثلاً نام دانشجو یا شناسه رکورد) و یک <b>اشاره‌گر به گره بعدی</b> (next) را ذخیره کند.</p>
</div>

In [1]:
# Define the Node class for the linked list
# Each node holds a key, its associated data, and a pointer (next)
class Node:
    def __init__(self, key, data):
        self.key = key       # The key (e.g., student ID)
        self.data = data     # The data (e.g., "R1", student name, etc.)
        self.next = None     # Pointer to the next node in the chain

# Test creating a node
node_test = Node(145, "R1")
print(f"Created Node with key: {node_test.key}, data: {node_test.data}, next: {node_test.next}")
node_test

Created Node with key: 145, data: R1, next: None


<div dir="rtl" align="justify">
  <h2>پیاده‌سازی در پایتون: بخش دوم - کلاس جدول هش (HashTable)</h2>
  <p>اکنون کلاس اصلی <code>HashTable</code> را می‌سازیم. این کلاس شامل موارد زیر خواهد بود:</p>
  <ul>
    <li><code>__init__(self, size)</code>: سازنده‌ای که جدول (آرایه) را با اندازه مشخص (در مثال ما 101) می‌سازد و آن را با مقادیر <code>None</code> پر می‌کند.</li>
    <li><code>_hash(self, key)</code>: متد خصوصی برای محاسبه تابع هش (<code>key % size</code>).</li>
    <li><code>insert(self, key, data)</code>: متدی برای درج داده. این متد ابتدا ایندکس را هش می‌کند. اگر خانه خالی بود، گره جدید را آنجا قرار می‌دهد. اگر خالی نبود (برخورد)، به انتهای لیست پیوندی در آن خانه می‌رود و گره جدید را اضافه می‌کند.</li>
    <li><code>search(self, key)</code>: متدی برای جستجوی یک کلید. ایندکس را هش می‌کند، سپس لیست پیوندی آن خانه را برای یافتن کلید مورد نظر پیمایش می‌کند.</li>
    <li><code>display(self)</code>: یک متد کمکی برای چاپ محتویات جدول به شکلی خوانا.</li>
  </ul>
</div>

In [2]:
# Define the HashTable class
class HashTable:
    def __init__(self, size):
        # Initialize the table with 'size' empty slots (None)
        self.size = size
        self.table = [None] * self.size

    def _hash(self, key):
        # The hash function, as shown in the example (mod 101)
        # We use self.size to be more general
        return key % self.size

    def insert(self, key, data):
        # Insert a new key-data pair
        index = self._hash(key)
        new_node = Node(key, data)

        if self.table[index] is None:
            # If the slot is empty, place the new node here
            # This node is the HEAD of the linked list at this index
            self.table[index] = new_node
        else:
            # If there's a collision, traverse to the end of the chain
            current = self.table[index]
            while current.next:
                current = current.next
            # Add the new node at the end of the chain
            current.next = new_node
            
    def search(self, key):
        # Search for a key in the hash table
        index = self._hash(key)
        current = self.table[index] # Get the head of the chain

        # Traverse the chain at this index
        while current:
            if current.key == key:
                # Found the key, return its data
                return current.data
            current = current.next
        
        # Key not found after traversing the entire chain (or if chain was empty)
        return None

    def display(self):
        # Utility function to print the hash table contents
        print("--- Hash Table Contents ---")
        print(f"Table Size: {self.size}")
        for i in range(self.size):
            if self.table[i] is not None:
                # This slot has a chain
                print(f"Index {i}: ", end="")
                current = self.table[i]
                chain = []
                while current:
                    chain.append(f"(Key={current.key}, Data={current.data})")
                    current = current.next
                print(" -> ".join(chain))
        print("-----------------------------")

# Test initializing the table
test_ht = HashTable(10) # Create a small test table
print(f"Created HashTable with size: {test_ht.size}")
test_ht

Created HashTable with size: 10


<div dir="rtl" align="justify">
  <h2>اجرای مثال نمونه</h2>
  <p>حال، داده‌های نمونه را در جدول هش خود درج میکنیم. ما یک جدول با اندازه 101 می‌سازیم.</p>
  <p>برای اینکه زنجیره‌ها به شکل خاصی (مثلاً R1 و سپس R5) ساخته شوند، ترتیب درج اهمیت دارد. ما فرض می‌کنیم <code>R1</code> قبل از <code>R5</code> و <code>R2</code> قبل از <code>R6</code> درج شده است.</p>
  <p>ترتیب درج داده‌ها:</p>
  <ol>
    <li><code>insert(145, "R1")</code> -> <code>h(145) = 44</code></li>
    <li><code>insert(17, "R2")</code>  -> <code>h(17) = 17</code></li>
    <li><code>insert(30, "R3")</code>  -> <code>h(30) = 30</code></li>
    <li><code>insert(15, "R4")</code>  -> <code>h(15) = 15</code></li>
    <li><code>insert(44, "R5")</code>  -> <code>h(44) = 44</code> (<b>برخورد!</b> به انتهای لیست در ایندکس 44 اضافه می‌شود)</li>
    <li><code>insert(118, "R6")</code> -> <code>h(118) = 17</code> (<b>برخورد!</b> به انتهای لیست در ایندکس 17 اضافه می‌شود)</li>
  </ol>
  <p>پس از درج، متد <code>display()</code> را فراخوانی می‌کنیم تا ساختار نهایی را ببینیم.</p>
</div>

In [3]:
# 1. Initialize the Hash Table with size 101
# This creates a list of 101 'None' values
student_hash_table = HashTable(101)

# 2. Insert the sample data
# The order is important to replicate the desired chains

# (R1, key=145) hashes to 145 % 101 = 44
student_hash_table.insert(145, "R1")

# (R2, key=17) hashes to 17 % 101 = 17
student_hash_table.insert(17, "R2")

# (R3, key=30) hashes to 30 % 101 = 30
student_hash_table.insert(30, "R3")

# (R4, key=15) hashes to 15 % 101 = 15
student_hash_table.insert(15, "R4")

# (R5, key=44) hashes to 44 % 101 = 44 (COLLISION with R1)
# Appends to the chain at index 44
student_hash_table.insert(44, "R5")

# (R6, key=118) hashes to 118 % 101 = 17 (COLLISION with R2)
# Appends to the chain at index 17
student_hash_table.insert(118, "R6")

# 3. Display the final hash table structure
# The output shows the final structure
student_hash_table.display()

--- Hash Table Contents ---
Table Size: 101
Index 15: (Key=15, Data=R4)
Index 17: (Key=17, Data=R2) -> (Key=118, Data=R6)
Index 30: (Key=30, Data=R3)
Index 44: (Key=145, Data=R1) -> (Key=44, Data=R5)
-----------------------------


<div dir="rtl" align="justify">
  <h2>تست جستجو (Search)</h2>
  <p>در نهایت، متد <code>search</code> را تست می‌کنیم تا مطمئن شویم به درستی کار می‌کند. ما موارد زیر را جستجو خواهیم کرد:</p>
  <ul>
    <li><b>جستجوی کلید 145:</b> باید رکورد <code>R1</code> را برگرداند (ابتدای زنجیره در ایندکس 44).</li>
    <li><b>جستجوی کلید 44:</b> باید رکورد <code>R5</code> را برگرداند (انتهای زنجیره در ایندکس 44).</li>
    <li><b>جستجوی کلید 118:</b> باید رکورد <code>R6</code> را برگرداند (انتهای زنجیره در ایندکس 17).</li>
    <li><b>جستجوی کلید 999:</b> این کلید وجود ندارد (<code>999 % 101 = 90</code>). خانه 90 خالی است، بنابراین باید <code>None</code> برگرداند.</li>
  </ul>
</div>

In [4]:
# 4. Test the search functionality

# Search for a key at the start of a chain (R1)
key_to_find = 145
result = student_hash_table.search(key_to_find)
print(f"Search for key {key_to_find}: Found data = {result}")

# Search for a key at the end of a chain (R5)
key_to_find = 44
result = student_hash_table.search(key_to_find)
print(f"Search for key {key_to_find}: Found data = {result}")

# Search for the other chain (R6)
key_to_find = 118
result = student_hash_table.search(key_to_find)
print(f"Search for key {key_to_find}: Found data = {result}")

# Search for a key that doesn't exist
key_to_find = 999
result = student_hash_table.search(key_to_find)
print(f"Search for key {key_to_find}: Found data = {result}")

Search for key 145: Found data = R1
Search for key 44: Found data = R5
Search for key 118: Found data = R6
Search for key 999: Found data = None
